# CREDIT CARD FRAUD DETECTION

The objective of this project is to create a Naive Bayes model capable of detecting fraud in credit card operations, thus seeking to minimize the risk and loss of the business.

IMPORING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [ ]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [ ]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57531 entries, 0 to 57530
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    57531 non-null  int64  
 1   V1      57531 non-null  float64
 2   V2      57531 non-null  float64
 3   V3      57531 non-null  float64
 4   V4      57531 non-null  float64
 5   V5      57530 non-null  float64
 6   V6      57530 non-null  float64
 7   V7      57530 non-null  float64
 8   V8      57530 non-null  float64
 9   V9      57530 non-null  float64
 10  V10     57530 non-null  float64
 11  V11     57530 non-null  float64
 12  V12     57530 non-null  float64
 13  V13     57530 non-null  float64
 14  V14     57530 non-null  float64
 15  V15     57530 non-null  float64
 16  V16     57530 non-null  float64
 17  V17     57530 non-null  float64
 18  V18     57530 non-null  float64
 19  V19     57530 non-null  float64
 20  V20     57530 non-null  float64
 21  V21     57530 non-null  float64
 22

In [ ]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        1
V6        1
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [ ]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    57371
1.0      159
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [ ]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [ ]:
print(legit.shape)
print(fraud.shape)

(57371, 31)
(159, 31)


In [ ]:
# statistical measures of the amount column
legit.Amount.describe()

count    57371.000000
mean        94.964936
std        258.837451
min          0.000000
25%          7.730000
50%         26.000000
75%         87.265000
max      12910.930000
Name: Amount, dtype: float64

In [ ]:
fraud.Amount.describe()

count     159.000000
mean       94.008491
std       226.356407
min         0.000000
25%         1.000000
50%         4.560000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64

In [ ]:
#Comparing values for legit and fraud transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,31182.415593,-0.219113,-0.022390,0.719137,0.156512,-0.245965,0.109222,-0.096562,0.046517,0.082463,...,0.046811,-0.030106,-0.105994,-0.038668,0.007397,0.135838,0.019652,0.001468,0.003733,94.964936
1.0,28211.811321,-7.202722,5.178029,-9.744764,5.629843,-5.323789,-2.182973,-7.596616,3.494864,-3.408036,...,0.440868,0.866669,-0.211629,-0.271093,-0.081439,0.238644,0.127501,0.573380,0.033555,94.008491


UNDER-SAMPLING OF DATA

Since the dataset is highly unbalanced, we perform undersampling of the data to build a sample dataset containing similar distribution of normal transactions and fraudulent transactions.

In [ ]:
legit_sample = legit.sample(n=492)

In [ ]:
new=pd.concat([legit_sample,fraud],axis=0)

In [ ]:
new.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
49648,44149,-0.413824,1.068785,1.393601,0.093227,0.029106,-0.697971,0.643290,0.053080,-0.396224,...,-0.214412,-0.529397,0.066324,0.343668,-0.281233,0.081837,0.260041,0.098093,7.18,0.0
56245,47332,-0.282569,0.271178,1.702742,0.041995,-0.129158,0.640892,-0.179153,0.416162,0.732239,...,0.013915,0.167187,-0.016313,-0.782556,-0.867386,0.252826,0.192451,0.178093,11.50,0.0
42189,40988,0.728921,-0.317696,0.212950,1.261416,-0.306467,-0.267181,0.405519,0.018963,-0.426203,...,-0.256662,-1.301015,0.091834,0.134100,0.023798,-0.859522,-0.019397,0.053315,214.00,0.0
18602,29632,-0.433639,0.784044,1.769839,-0.169176,0.106752,-0.189486,0.709830,0.032837,-0.398000,...,-0.191182,-0.375916,0.015258,0.179937,-0.376190,0.044973,0.105523,-0.114779,13.57,0.0
53472,46000,-1.181871,-0.123701,2.497943,0.456774,0.420220,-0.711575,-0.261514,0.203062,0.329290,...,0.110312,0.232312,-0.007974,0.375832,-0.054772,-0.590944,0.132929,0.158213,9.99,0.0


In [ ]:
new.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
53794,46149,-1.346509,2.132431,-1.854355,2.116998,-1.070378,-1.092671,-2.230986,1.036425,-1.895516,...,0.609508,0.202874,-0.060791,-0.186733,-0.017401,-0.283751,0.395451,0.233139,1.00,1.0
55401,46925,-0.481531,1.059542,0.647117,0.905586,0.819368,-0.091184,0.504135,0.161064,-0.765054,...,0.070633,0.192491,-0.174659,-0.438908,0.239259,-0.217823,-0.072852,0.010463,1.00,1.0
56703,47545,1.176716,0.557091,-0.490800,0.756424,0.249192,-0.781871,0.228750,-0.040840,-0.432111,...,-0.062166,-0.128168,-0.040176,0.110040,0.437891,0.368809,-0.018287,0.031173,0.76,1.0
57248,47826,-0.887287,1.390002,1.219686,1.661425,1.009228,-0.733908,0.855829,0.000077,-1.275631,...,-0.083734,-0.346930,-0.050619,0.231044,-0.450760,-0.376205,0.034504,0.157775,7.58,1.0
57470,47923,0.364377,1.443523,-2.220907,2.036985,-1.237055,-1.728161,-2.058582,0.358895,-1.393306,...,0.402730,-0.132129,-0.032977,0.460861,0.560404,0.409366,0.539668,0.296918,0.76,1.0


In [ ]:
#Counting values of both classes in the undersampled dataset
new['Class'].value_counts()

0.0    492
1.0    159
Name: Class, dtype: int64

In [ ]:
new.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,30894.132114,-0.154807,0.008633,0.765681,0.087180,-0.250724,0.075774,-0.076574,0.042236,0.149337,...,0.060707,-0.039988,-0.121158,-0.048773,0.042406,0.117154,0.039872,-0.009237,0.012616,90.828394
1.0,28211.811321,-7.202722,5.178029,-9.744764,5.629843,-5.323789,-2.182973,-7.596616,3.494864,-3.408036,...,0.440868,0.866669,-0.211629,-0.271093,-0.081439,0.238644,0.127501,0.573380,0.033555,94.008491


SELECTING THE FEATURE AND TARGET COLUMNS 

In [ ]:
#Feature columns
X = new.drop(columns='Class', axis=1)

#Target columns 
Y = new['Class']

In [ ]:
print(X)

        Time        V1        V2        V3        V4        V5        V6  \
49648  44149 -0.413824  1.068785  1.393601  0.093227  0.029106 -0.697971   
56245  47332 -0.282569  0.271178  1.702742  0.041995 -0.129158  0.640892   
42189  40988  0.728921 -0.317696  0.212950  1.261416 -0.306467 -0.267181   
18602  29632 -0.433639  0.784044  1.769839 -0.169176  0.106752 -0.189486   
53472  46000 -1.181871 -0.123701  2.497943  0.456774  0.420220 -0.711575   
...      ...       ...       ...       ...       ...       ...       ...   
53794  46149 -1.346509  2.132431 -1.854355  2.116998 -1.070378 -1.092671   
55401  46925 -0.481531  1.059542  0.647117  0.905586  0.819368 -0.091184   
56703  47545  1.176716  0.557091 -0.490800  0.756424  0.249192 -0.781871   
57248  47826 -0.887287  1.390002  1.219686  1.661425  1.009228 -0.733908   
57470  47923  0.364377  1.443523 -2.220907  2.036985 -1.237055 -1.728161   

             V7        V8        V9  ...       V20       V21       V22  \
49648  0.6432

In [ ]:
print(Y)

49648    0.0
56245    0.0
42189    0.0
18602    0.0
53472    0.0
        ... 
53794    1.0
55401    1.0
56703    1.0
57248    1.0
57470    1.0
Name: Class, Length: 651, dtype: float64


SPLITTING DATASET INTO TRAIN AND TEST DATA

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.21, stratify=Y, random_state=48)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(651, 30) (514, 30) (137, 30)


# MODEL TRAINING

Since this is a binary classification problem, we will use Logistic Regression as our model. 

In [ ]:
model=model = GaussianNB()

In [ ]:
# training the Naive Bayes Model with Training Data
model.fit(X_train,Y_train)

GaussianNB()

# MODEL EVALUATION

After fitting our model on training data, we will now check the accuracy of the model by using test data. 

In [ ]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9649805447470817


In [ ]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.948905109489051
